### Further Reading: ctypes demo:  struct，one and two-dimensional array

`extern "C"` makes it possible to link with functions written in the C programming language, and to define, in a C++ program, functions that can be called from the modules written in C.

Reference operator (&) and Dereference operator (*)

In [8]:
%%file ./demo/src/ctypemore.cpp

// 1 Struct
struct SimpleStruct
{
    int nNo;
    float fValue;
};

extern "C"  __declspec(dllexport)  int __stdcall  TestSimpleStruct(int *n,int &m,SimpleStruct struin,SimpleStruct *struout)
{
  struout->fValue= struin.fValue+2;
  struout->nNo=struin.nNo+3;
  
  *n=struin.nNo+20;
  m=*n+30;
  return struout->nNo;
}

// 2 using the name of one-dimensional array in Python.non byref
extern "C"  __declspec(dllexport)  void  __stdcall  TestArray1(int nsize,double *narray)
{
  for(int i=0; i<nsize; i++)
  {
     narray[i]=i*2.3;  
  }
}

// 3 using the name of two-dimensional array in Python.  non byref
extern "C"  __declspec(dllexport)  void  __stdcall  TestArray21(int ni, int nj,double *ptr)
{
    int i, j;
    for(i=0; i<ni; i++)
    {
        for(j=0; j<nj; j++)
        {
            ptr[i*ni+j]= ptr[i*ni+j]+i*ni + j;
        }
	}       
}

// 4 using byref in Python
extern "C"  __declspec(dllexport)  void  __stdcall  TestArray22(int ni, int nj,double **ptr)
{
    int i, j;
    for(i=0; i<ni; i++)
    {
        for(j=0; j<nj; j++)
        {
            ptr[i][j]=ptr[i][j]+i*ni + j;
        }
	} 
}

Writing ./demo/src/ctypemore.cpp


In [14]:
%%file ./demo/makefile-ctypemoredll

all: libctypemoredll

libctypemoredll: obj
	 g++  -shared -o ./demo/bin/libctypemore.dll ctypemore.o
	 del ctypemore.o

obj: 
	 g++ -c -O3 -Wall -fPIC  ./demo/src/ctypemore.cpp 

Overwriting ./demo/makefile-ctypemoredll


In [16]:
!make -f ./demo/makefile-ctypemoredll

g++ -c -O3 -Wall -fPIC  ./demo/src/ctypemore.cpp 
g++  -shared -o ./demo/bin/libctypemore.dll ctypemore.o
del ctypemore.o


In [17]:
from ctypes import *

class SimpStruct(Structure):
    _fields_ = [("nNo", c_int),
                ("fValue", c_float)]

Objdll=windll.LoadLibrary("./demo/bin/libctypemore.dll")

# 1 TestSimpleStruct
f1=Objdll.TestSimpleStruct
simplein = SimpStruct()
simplein.nNo = 16
simplein.fValue = 3.14

simpleout = SimpStruct()

n=c_int()
m=c_int()
nNo=f1(byref(n),byref(m),simplein,byref(simpleout))

print(simpleout.fValue)
print(nNo)
print(n.value)
print(m.value)

# 2  one-dimensional array
f2=Objdll.TestArray1
na=c_int()
na=10;

narray=(c_double*10)() # must (c_double*10)(),not c_double*10()

# non byref，using the name of one-dimensional array
f2(na,narray)
print(narray[8])

# 3 two-dimensional array  - 1
f3=Objdll.TestArray21
ptr = (c_double*5*4)()
# two-dimensional array: j*i
for i in range(4):
    for j in range(5):
        ptr[i][j] = i + j
# non byref，using the name of two-dimensional array
f3(4,5,ptr)
print(ptr[1][3])

# 4 two-dimensional array-2
f4=Objdll.TestArray22
indata = (POINTER(c_double) * 5)()
for i in range(5):
    # Allocate arrays of double
    indata[i] = (c_double * 6)()
    for j in range(6):
        indata[i][j] = 1.0

f4(5,6,byref(indata))
print(indata[1][3])


5.140000343322754
19
36
66
18.4
20.0
9.0


### Further reading

ctypes http://docs.python.org/3/library/ctypes.html

C-Types Foreign Function Interface (numpy.ctypeslib)
  https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html
  